In [53]:
import pandas as pd

In [54]:
df = pd.read_json('data/core_data.jsonl', lines=True)
df.head()

,instruction,instances
0,You will be given a series of words. Output th...,[{'instruction_with_input': 'You will be given...
1,"In this task, you're given a short description...","[{'instruction_with_input': 'In this task, you..."
2,The input is a list of words. Your goal is to ...,[{'instruction_with_input': 'The input is a li...
3,You need to read the text and answer whether i...,[{'instruction_with_input': 'You need to read ...
4,You will be given two pieces of text. The firs...,[{'instruction_with_input': 'You will be given...


In [55]:
df['instances_length'] = df['instances'].apply(len)
df[df['instances_length'] != 1]

,instruction,instances,instances_length
36,You will be given a piece of text either about...,[{'instruction_with_input': 'You will be given...,2
40,"You will be given two strings, A and B. Print ...",[{'instruction_with_input': 'You will be given...,5
65,You will be given a piece of text either about...,[{'instruction_with_input': 'You will be given...,8
207,You need to answer the question 'Are the given...,[{'instruction_with_input': 'You need to answe...,8
259,Determine whether the statement is true or false.,[{'instruction_with_input': 'Determine whether...,12
...,...,...,...
63149,"In this task, you are given a list of words an...","[{'instruction_with_input': 'In this task, you...",2
63735,You are given a scientific question (difficult...,[{'instruction_with_input': 'You are given a s...,2
63980,"In this task, you are given an incomplete sent...","[{'instruction_with_input': 'In this task, you...",2
64147,"You are given an incomplete dialogue, and your...",[{'instruction_with_input': 'You are given an ...,2


In [47]:
df[df['instances_length'] != 1]['instances'].iloc[0]

[{'instruction_with_input': "You will be given a piece of text either about an everyday event, or a general statement. If the event seems impossible for you, or the general statement does not make sense matches your commonsense, output 'False', otherwise output 'True'.\nText: A man went to buy some groceries and came back with a new car.",
  'input': 'Text: A man went to buy some groceries and came back with a new car.',
  'constraints': "The output should be one of the two: 'True' or 'False'.",
  'output': 'False'},
 {'instruction_with_input': "You will be given a piece of text either about an everyday event, or a general statement. If the event seems impossible for you, or the general statement does not make sense matches your commonsense, output 'False', otherwise output 'True'.\nText: It was raining so hard that the water in the ocean boiled.",
  'input': 'Text: It was raining so hard that the water in the ocean boiled.',
  'constraints': "The output should be one of the two: 'True

In [57]:
df_exploded = df.explode('instances')
df_exploded

,instruction,instances,instances_length
0,You will be given a series of words. Output th...,{'instruction_with_input': 'You will be given ...,1
1,"In this task, you're given a short description...","{'instruction_with_input': 'In this task, you'...",1
2,The input is a list of words. Your goal is to ...,{'instruction_with_input': 'The input is a lis...,1
3,You need to read the text and answer whether i...,{'instruction_with_input': 'You need to read t...,1
4,You will be given two pieces of text. The firs...,{'instruction_with_input': 'You will be given ...,1
...,...,...,...
66005,"In this task, you are given two sentences take...","{'instruction_with_input': 'In this task, you ...",1
66006,"In this task, you're given three strings and y...","{'instruction_with_input': 'In this task, you'...",1
66007,Given two strings consisting of alphabetic cha...,{'instruction_with_input': 'Given two strings ...,1
66008,You are given several pieces of information an...,{'instruction_with_input': 'You are given seve...,1


In [49]:
df_exploded['instances'].iloc[0].keys()

dict_keys(['instruction_with_input', 'input', 'constraints', 'output'])

In [50]:
def get_prompt(row):
    inst_dict = row['instances']
    input_text = f"input: {inst_dict['input']},\ninstruction_with_input: {inst_dict['instruction_with_input']},\nconstraints: {inst_dict['constraints']}"
    prompt = f"<s>[INST] {input_text} [/INST]"
    return prompt

def get_output(row):
    return row['instances']['output']


df_exploded['prompt'] = df_exploded.apply(get_prompt, axis=1)
df_exploded['output'] = df_exploded.apply(get_output, axis=1)

df_with_prompt = df_exploded.drop('instances_length', axis=1)

df_with_prompt.head()

,instruction,instances,prompt,output
0,You will be given a series of words. Output th...,{'instruction_with_input': 'You will be given ...,"<s>[INST] input: Words: ['Hello', 'world'].,\n...",world\nHello
1,"In this task, you're given a short description...","{'instruction_with_input': 'In this task, you'...",<s>[INST] input: Description: A man walks into...,The man walks into the bar.\nHe orders a drink...
2,The input is a list of words. Your goal is to ...,{'instruction_with_input': 'The input is a lis...,"<s>[INST] input: Words: ['b', 'a', 'n', 'a', '...",Yes
3,You need to read the text and answer whether i...,{'instruction_with_input': 'You need to read t...,<s>[INST] input: Text: The balloon floated up ...,Up
4,You will be given two pieces of text. The firs...,{'instruction_with_input': 'You will be given ...,<s>[INST] input: Description/event: I put some...,No


In [51]:
print(df_with_prompt['prompt'].iloc[0])

<s>[INST] input: Words: ['Hello', 'world'].,
instruction_with_input: You will be given a series of words. Output these words in reverse order, with each word on its own line.
Words: ['Hello', 'world'].,
constraints: None. [/INST]


In [52]:
# Assuming `df` is your DataFrame
df_with_prompt.to_json('data/processed_data.jsonl', orient='records', lines=True)